In [1]:
import getpass

user = getpass.getuser()
from omni.isaac.kit import SimulationApp

# Set the path below to your desired nucleus server
# Make sure you installed a local nucleus server before this
# simulation_app = SimulationApp({"livesync_usd": f"omniverse://localhost/Users/{user}/demo_raycast.usd"})
# simulation_app = SimulationApp({"livesync_usd": f"omniverse://localhost/Users/{user}/pre-world.usd"})
# simulation_app = SimulationApp({"livesync_usd": f"omniverse://localhost/Users/{user}/texture_sandbox.usd"})
simulation_app = SimulationApp({"livesync_usd": f"omniverse://localhost/Users/{user}/power_thermal_demo.usd"})

[Warning] [omni.isaac.kit] 
[Warning] [omni.isaac.kit] =========================== DEPRECATION WARNING ===========================
[Warning] [omni.isaac.kit] 
[Warning] [omni.isaac.kit] Traceback (most recent call last):
[Warning] [omni.isaac.kit] File '/home/ljb/.local/share/ov/pkg/isaac-sim-4.1.0/kit/python/lib/python3.10/runpy.py', line 196
[Warning] [omni.isaac.kit]   return _run_code(code, main_globals, None,
[Warning] [omni.isaac.kit] File '/home/ljb/.local/share/ov/pkg/isaac-sim-4.1.0/kit/python/lib/python3.10/runpy.py', line 86
[Warning] [omni.isaac.kit]   exec(code, run_globals)
[Warning] [omni.isaac.kit] File '/home/ljb/.local/share/ov/pkg/isaac-sim-4.1.0/kit/python/lib/python3.10/site-packages/ipykernel_launcher.py', line 18
[Warning] [omni.isaac.kit]   app.launch_new_instance()
[Warning] [omni.isaac.kit] File '/home/ljb/.local/share/ov/pkg/isaac-sim-4.1.0/kit/python/lib/python3.10/site-packages/traitlets/config/application.py', line 1075
[Warning] [omni.isaac.kit]   app.sta

kit.widget.filter-1.1.4] startup
[1.768s] [ext: omni.graph.core-2.170.3] startup
[1.771s] [ext: omni.kit.window.extensions-1.4.9] startup
[1.777s] [ext: omni.kit.hotkeys.core-1.3.3] startup
[1.777s] [ext: omni.kit.context_menu-1.8.0] startup
[1.779s] [ext: omni.mdl-52.0.1] startup
[1.796s] [ext: omni.iray.libs-0.0.0] startup
[1.800s] [ext: omni.kit.primitive.mesh-1.0.16] startup
[1.803s] [ext: omni.kit.stage_template.core-1.1.21] startup
[1.803s] [ext: omni.mdl.neuraylib-0.2.5] startup
[1.804s] [ext: omni.ujitso.processor.texture-1.0.0] startup
[1.805s] [ext: omni.kit.window.file_exporter-1.0.29] startup
[1.807s] [ext: omni.kit.stage_templates-1.2.3] startup
[1.809s] [ext: omni.volume-0.5.0] startup
[1.811s] [ext: omni.ujitso.client-0.0.0] startup
[1.812s] [ext: omni.kit.window.file_importer-1.1.11] startup
[1.813s] [ext: omni.kit.window.drop_support-1.0.2] startup
[1.813s] [ext: omni.kit.window.file-1.3.52] startup
[1.816s] [ext: omni.kit.window.content_browser_registry-0.0.6] startup

Warp 1.2.1 initialized:
   CUDA Toolkit 11.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 3060" (12 GiB, sm_86, mempool enabled)
   Kernel cache:
     /home/ljb/.cache/warp/1.2.1
[2.927s] [ext: omni.ui_query-1.1.2] startup
[2.929s] [ext: omni.fabric.commands-1.1.4] startup
[2.937s] [ext: omni.physx.vehicle-106.0.20] startup
[2.958s] [ext: omni.kit.manipulator.prim.core-107.0.3] startup
[2.967s] [ext: omni.kit.graph.editor.core-1.5.3] startup
[2.970s] [ext: omni.kit.ui_test-1.2.18] startup
[2.972s] [ext: omni.kit.graph.usd.commands-1.3.1] startup
[2.974s] [ext: omni.physx.camera-106.0.20] startup
[2.984s] [ext: omni.kit.manipulator.prim.fabric-106.0.1] startup
[2.987s] [ext: omni.kit.manipulator.prim.usd-106.0.1] startup
[2.989s] [ext: omni.physx.cct-106.0.20] startup
[3.002s] [ext: omni.physics.tensors-106.0.20] startup
[3.014s] [ext: omni.isaac.version-1.1.0] startup
[3.019s] [ext: omni.kit.manipulator.selection-104.0.9] startup
[3.023s] [e

In [2]:
import carb
from omni.physx import get_physx_scene_query_interface
from omni.isaac.core import World
from omni.isaac.core.objects import DynamicCuboid, DynamicSphere, DynamicCone
from omni.timeline import get_timeline_interface
import numpy as np
import matplotlib.pyplot as plt
import omni.ui
from pxr import Kind, Sdf, Usd, UsdGeom, UsdShade
from omni.isaac.core.utils.stage import get_current_stage
import omni.isaac.core.utils.prims as prims_utils

In [3]:
import omni.isaac.core.utils.mesh as mesh_utils
import omni.isaac.core.utils.rotations as rotations_utils

In [4]:
world = World(stage_units_in_meters=1.0)
stage = get_current_stage()

In [5]:
world.clear()
# world.scene.add_default_ground_plane()
# A render/ step or an update call is needed to reflect the changes to the opened USD in Isaac Sim GUI
#  Note: avoid pressing play/ pause or stop in the GUI in this workflow.
world.render()

In [6]:
# Create the Sun
sun = prims_utils.create_prim(
    "/World/Sun",
    "SphereLight",
    position=np.array([1.0, 1.0, 1.0]),
    attributes={
        "inputs:radius": 1,
        "inputs:intensity": 5e3,
        "inputs:color": (1.0, 1.0, 1.0)
    }
)
world.render()

In [7]:
def adjust_light(stage, position):
    lights = []
    for prim in stage.Traverse():
        if prim.GetTypeName() == "SphereLight":
            lights.append(prim)
    if lights:
        light = lights[0]
        light_path = light.GetPath()
        # Adjust position
        translate_attr = stage.GetPrimAtPath(light_path).GetAttribute("xformOp:translate")
        translate_attr.Set(position)
        # Adjust intensity
        intensity_attr = stage.GetPrimAtPath(light_path).GetAttribute("inputs:intensity")
        intensity_attr.Set(3e9)
        # Adjust radius
        radius_attr = stage.GetPrimAtPath(light_path).GetAttribute("inputs:radius")
        radius_attr.Set(.01)

In [10]:
# Define Sun position
# sun_coord = (5, 5, 5)
# sun_coord = (-5, -5, 5)
sun_coord = (8, -5, 3)
# sun_coord = (-5, 5, 5)

adjust_light(stage, sun_coord)
world.render()

In [11]:
# Load a usd prepared with the meshes we need
from omni.isaac.core.utils.prims import create_prim
from omni.isaac.core.prims.xform_prim import XFormPrim


usd_path = "../../../lunar_base.usd"
prim_path = "/World/Lunar_Base"

ground_path = prim_path + "/Ground"
panel_path = prim_path + "/Solar_Generator/Panel"
support1_path = prim_path + "/Solar_Generator/Support1"
support2_path = prim_path + "/Solar_Generator/Support2"
all_paths = [ground_path, panel_path, support1_path, support2_path]

In [12]:
prim = create_prim(prim_path=prim_path, usd_path=usd_path)
xform_prim = XFormPrim(prim.GetPath().pathString)
world.render()

In [13]:
#for path in all_paths:
#    prims_utils.set_prim_attribute_value(path, "primvars:displayColor", face_colors)
#world.render()

In [14]:
# isaacsim bug:
# prims_utils.get_prim_attribute_value(panel_path, "xformOp:orient")

In [15]:
timeline = get_timeline_interface()
timeline.play()
for _ in range(10):
    simulation_app.update()
timeline.pause()

In [16]:
def set_prim_orientation_euler(path, angles, degrees=False):
    prims_utils.set_prim_attribute_value(
        path, 
        "xformOp:orient", 
        rotations_utils.euler_angles_to_quat(angles, degrees=degrees)
    )

## Compute power generation

In [17]:
def get_mesh_vertices_world(path):
    mesh_prim = stage.GetPrimAtPath(path)
    world_prim = stage.GetPrimAtPath("/World")
    points_world = mesh_utils.get_mesh_vertices_relative_to(mesh_prim, world_prim)
    return points_world

In [18]:
def get_quad_centers(points_coord, face_vertex_indices):
    num_quads = len(face_vertex_indices) // 4
    quad_centers = np.array([
        np.mean(points_coord[
                list(face_vertex_indices[i*4:(i+1)*4])
                ], axis=0) 
        for i in range(num_quads)
    ])
    return quad_centers

In [19]:
def compute_sun_to_mesh(mesh_face_centers, path_sun="/World/Sun", output='dotproducts'):
    sun_coord = prims_utils.get_prim_attribute_value(path_sun, "xformOp:translate")
    
    dot_products = np.zeros(len(mesh_face_centers))
    hit_distances = []  # for debug

    for i, center in enumerate(mesh_face_centers):
        vector_to_cell = center - sun_coord
        distance = np.linalg.norm(vector_to_cell)
        direction = vector_to_cell / distance
    
        # Cast a ray from the Sun to the cell center
        hit_info = get_physx_scene_query_interface().raycast_closest(sun_coord, direction, distance + 1)  # increase max distance to avoid ray not reaching the surface
    
        if hit_info["hit"]:
            hit_distance = np.linalg.norm(hit_info["position"] - center)
            hit_distances.append(hit_distance)
            if hit_distance < 0.001: # consider that there is no obstruction between the sun and the cell center
                normal = np.array(hit_info["normal"])
                dot_products[i] = np.dot(normal, direction)
            else: # cell is in shadow of obstruction
                dot_products[i] = np.nan
        else:
            print(f"no hit, angle for cell center at {center} is undefined")
            dot_products[i] = np.nan  # No hit, angle is undefined
    
    if output=='dotproducts':
        return dot_products
    elif output=='angles_deg':
        sun_angles = np.arccos(dot_products) * (180 / np.pi)  # Convert to degrees
        return sun_angles
    else:
        print("possible outputs are only 'dotproducts' or 'angles_deg'") 

In [44]:
def compute_power(mesh_path, dot_products):
    sun_power = 1376     # w/m2
    solar_cell_eff = .3  # 30%
    length, width, height = prims_utils.get_prim_attribute_value(mesh_path, "xformOp:scale")
    cell_area = length * width / 100    
    power_generation = sun_power * cell_area * np.abs(dot_products) * solar_cell_eff
    return power_generation

def color_conversion(array):
    # adjust for visualization
    cleaned = np.nan_to_num(array)  # replace all NaN by 0
    #normalized = np.interp(cleaned, [cleaned.min(), cleaned.max()], [0, 1])
    normalized = np.interp(power_generation, [0, 2], [0, 1])
    import matplotlib.cm as cm
    colors = cm.jet(normalized)[:, :3]

    # add two additional zeros at the end to match the shape required, not sure why
    colors = np.vstack([colors, np.zeros((2, 3))])
    return colors

In [45]:
# get point centers
face_vertex_indices = prims_utils.get_prim_attribute_value(panel_path, "faceVertexIndices")
points_world = get_mesh_vertices_world(panel_path)

quad_centers = get_quad_centers(points_world, face_vertex_indices)

In [46]:
dot_products = compute_sun_to_mesh(quad_centers)
power_generation = compute_power(panel_path, dot_products)
power_colors = color_conversion(power_generation)

In [47]:
prims_utils.set_prim_attribute_value(panel_path, "primvars:displayColor", power_colors)
world.render()

In [67]:
N = 100
panel_angles = np.zeros((N, 3)) 
panel_angles[:,0] = np.linspace(-50, 50, num=N)

sun_angles = np.zeros((N, 3))
sun_angles[:,0] = 5 * np.cos(np.linspace(0, 3.14, num=N))
sun_angles[:,1] = 5 * np.sin(np.linspace(0, 3.14, num=N))
sun_angles[:,2] = np.linspace(.5, 5, num=N)

In [69]:
for p,s in zip(panel_angles, sun_angles):
    set_prim_orientation_euler(panel_path, p, degrees=True)
    adjust_light(stage, tuple(s))
    
    timeline.play()
    for _ in range(5):
        simulation_app.update()
    timeline.pause()

    face_vertex_indices = prims_utils.get_prim_attribute_value(panel_path, "faceVertexIndices")
    points_world = get_mesh_vertices_world(panel_path)
    quad_centers = get_quad_centers(points_world, face_vertex_indices)

    dot_products = compute_sun_to_mesh(quad_centers)
    power_generation = compute_power(panel_path, dot_products)
    power_colors = color_conversion(power_generation)

    prims_utils.set_prim_attribute_value(panel_path, "primvars:displayColor", power_colors)
    world.render()

## Utils

In [11]:
# each cube face is a different color:
n_points = len(prims_utils.get_prim_attribute_value(ground_path, "points"))

import matplotlib.colors as mcolors
# Define six equally spaced hues in the HSV space
hues = np.linspace(0, 1, 6, endpoint=False)  # hue = colorhweel: hue 0 and hue 1 are the same so do not include endpoint
rgb_colors = [mcolors.hsv_to_rgb((hue, 1.0, 1.0)) for hue in hues]
 
face_colors = np.zeros((n_points, 3))

# Define the index ranges based on the mesh point structure
assert n_points >= 280
cube_mask = [
    np.arange(0, 100),   # front
    np.arange(100, 200), # back
    np.arange(200, 220), # four sides
    np.arange(220, 240),
    np.arange(240, 260),
    np.arange(260, 280)
]

for i, mask in enumerate(cube_mask):
    face_colors[mask] = rgb_colors[i]

In [ ]:
# compute power generation

# Initialize the angle map
sun_angles = np.zeros(len(quad_centers))
dot_products = np.zeros(len(quad_centers))
hit_distances = []
for i, center in enumerate(quad_centers):
    vector_to_cell = center - sun_coord
    distance = np.linalg.norm(vector_to_cell)
    direction = vector_to_cell / distance

    # Cast a ray from the Sun to the cell center
    hit_info = get_physx_scene_query_interface().raycast_closest(sun_coord, direction, distance + 1)  # increase max distance to avoid ray not reaching the surface

    if hit_info["hit"]:
        hit_distance = np.linalg.norm(hit_info["position"] - center)
        hit_distances.append(hit_distance)
        if hit_distance < 0.001: # consider that there is no obstruction between the sun and the cell center
            normal = np.array(hit_info["normal"])
            dot_products[i] = np.dot(normal, direction)
            sun_angles[i] = np.arccos(dot_products[i]) * (180 / np.pi)  # Convert to degrees
        else: # cell is in shadow of obstruction
            sun_angles[i] = np.nan
    else:
        print(f"no hit, angle for cell center at {center} is undefined")
        sun_angles[i] = np.nan  # No hit, angle is undefined

sun_angles